In [1]:
import pandas as pd

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Add project root to sys.path for module imports
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [3]:
from src.utils.data_processing import create_train_and_test_datasets
from src.utils.models import MLP
from src.utils.train_functions import train, evaluate

# Load data

## Load parameter data

In [86]:
bp_55 = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/building_parameters/building_parameters_0_55.csv")
bp_2090 = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/building_parameters/building_parameters_55_2090.csv")
bp_4196 = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/building_parameters/building_parameters_2090_4196.csv")
bp_8209 = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/building_parameters/building_parameters_4196_8209.csv")

bp_55.set_index("building_id", inplace=True)
bp_2090.set_index("building_id", inplace=True)
bp_4196.set_index("building_id", inplace=True)
bp_8209.set_index("building_id", inplace=True)

In [90]:
# Merge all dataframes
building_parameters = pd.concat([bp_55, bp_2090, bp_4196, bp_8209], axis=0)
building_parameters.drop(columns=["Unnamed: 0"], inplace=True)

## ResStock data

In [91]:
processed_resstock = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/resstock/resstock_processed.csv")
processed_resstock.set_index("bldg_id", inplace=True)

In [92]:
# only include resstock rows whose index is in building_parameters
processed_resstock = processed_resstock[processed_resstock.index.isin(building_parameters.index)]

# Create datasets

In [93]:
train_dataset, test_dataset = create_train_and_test_datasets(
    features=processed_resstock,
    labels=building_parameters,
    test_size=0.2,
)

In [94]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define model

In [95]:
# HYPERPARAMETERS
# model hyperparameters
input_size = len(processed_resstock.columns)
hidden_sizes = [32, 64, 16]
output_size = len(building_parameters.columns)

# training hyperparameters
num_epochs = 50
learning_rate = 0.001

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [96]:
parameter_prediction_model = MLP(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
).to(device)

In [97]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(parameter_prediction_model.parameters(), lr=learning_rate)

# Train model

In [98]:
model_name = "parameter_prediction_model_1.pth"
writer = SummaryWriter(log_dir=f"/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/runs/{model_name.replace('.pth', '')}")

In [99]:
train(
    model=parameter_prediction_model,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    writer=writer,
    verbose=True
)

# Evaluate model

In [100]:
eval_loss = evaluate(
    model=parameter_prediction_model,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
)

In [101]:
eval_loss

0.03997972818951194